In [2]:
import numpy as np
import tensorflow as tf
import gym

In [3]:
env=gym.make('CartPole-v1')
obs=env.reset()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [4]:
def discounted_rewards(rewards_total,rewards,discount_rate):
    i=len(rewards)-1
    discounted_value=0
    while i>=0:
        rewards[i]+=discounted_value
        discounted_value=rewards[i]*discount_rate
        i-=1
    i=0
    while i<len(rewards):
        rewards_total.append(rewards[i])
        i+=1
        
def normalized_rewards(rewards_total):
    temp=np.array(rewards_total)
    Mean=np.mean(temp)
    std=np.std(temp)
    for i in range(len(rewards_total)):
        rewards_total[i]=(rewards_total[i]-Mean)/std
    return rewards_total

In [ ]:
input_size=4
learning_rate=0.01
discount_rate=0.95
initializer = tf.contrib.layers.variance_scaling_initializer()
X=tf.placeholder(tf.float32,shape=[1,input_size],name='X')
hidden=tf.contrib.layers.fully_connected(X,10,activation_fn=tf.nn.elu,weights_initializer=initializer)
final=tf.contrib.layers.fully_connected(hidden,1,activation_fn=None,weights_initializer=initializer)
output_probability=tf.nn.sigmoid(final)
x=tf.concat([output_probability,1-output_probability],axis=1)
action=tf.multinomial(tf.log(x),1)
y=1-tf.to_float(action)
error=tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=final)
optimizer=tf.train.AdamOptimizer(learning_rate)
grads_and_vars=optimizer.compute_gradients(error)
gradients = [grad for grad, variable in grads_and_vars]

gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
training_op = optimizer.apply_gradients(grads_and_vars_feed)
init = tf.global_variables_initializer()

In [ ]:
obs=env.reset()
Input=obs.reshape(1,input_size)
with tf.Session() as sess:
        init.run()
        step=0
        while(step<1000):
            rewards_total=[]
            gradients_total=[]
            episodes=10
            while episodes>0:
                obs=env.reset()
                done=False
                rewards_episode=[]
                i=0       
                while i<1000 and done==False:
                    A,Gradients= sess.run([action,gradients],feed_dict={X:Input})
                    gradients_total.append(Gradients)
                    obs,reward,done,info=env.step(A[0][0])
                    Input=obs.reshape(1,input_size)
                    rewards_episode.append(reward)
                    i+=1
                print ("Balanced for ",i,"steps")
                discounted_rewards(rewards_total,rewards_episode,discount_rate)
                episodes-=1
            rewards_total=normalized_rewards(rewards_total)
            Feed_dict=np.mean(np.multiply(np.array(gradients_total).T,np.array(rewards_total)),axis=1)
            sess.run(training_op,feed_dict={gradient_placeholders[0]:Feed_dict[0],gradient_placeholders[1]:Feed_dict[1],gradient_placeholders[2]:Feed_dict[2],gradient_placeholders[3]:Feed_dict[3]})
            step+=1
        sess.close()